In [1]:
from datasets import load_dataset
import pandas as pd
from nltk import pos_tag
import nltk

## Word Dictionaries
- no frequency check

### English

In [ ]:
## English
# nltk.download('averaged_perceptron_tagger_eng', download_dir='/work/hyujang/miniconda3/envs/thesis/nltk_data') # for en-pos tagging
nltk.download('wordnet', download_dir='/work/hyujang/miniconda3/envs/thesis/nltk_data') # en vocabulary
# en_ds = load_dataset("manu/project_gutenberg", split="en", streaming=True)
# en_ds = load_dataset("deepmind/pg19", split=None, streaming=True, trust_remote_code=True)
# en_ds = load_dataset("cambridge-climb/BabyLM", split=None, streaming=True, trust_remote_code=True) # smaller size
# en_ds = load_dataset("marksverdhei/wordnet-definitions-en-2021")
# en_df = en_ds['train'].to_pandas()
# data_list = list(en_ds.take(100))

from nltk.corpus import wordnet
noun_synsets = list(wordnet.all_synsets(pos=wordnet.NOUN))
en_nouns = set(lemma.name() for synset in noun_synsets for lemma in synset.lemmas() if "_" not in lemma.name())
print(len(en_nouns))

### Korean

In [7]:
## Korean
# ds = load_dataset("wicho/kor_3i4k")
# ds = load_dataset("binjang/NIKL-korean-english-dictionary")
import sys
# sys.setdefaultencoding("utf-8")
ds = load_dataset("hyunwoongko/korean-word-dict")
ko_dict = ds['train'].to_pandas()
# ds = load_dataset("hac541309/woori_spring_dict")
# df = ds['train'].to_pandas()
# df_split = df['text'].str.split(", ", expand=True) # takes too long

# from konlpy.tag import Okt # TODO: Mecab
# import unicodedata
# from tqdm import tqdm
# tqdm.pandas()
# okt = Okt()
# ko_dict['word'] = ko_dict['word'].progress_apply(lambda x: unicodedata.normalize("NFC", x)) # ㄴㅐ -> 내
# ko_dict['pos_okt'] = ko_dict['word'].progress_apply(lambda x: okt.pos(x))


In [12]:
from kiwipiepy import Kiwi
from tqdm import tqdm  # Make sure tqdm is installed and imported

kiwi = Kiwi()
ko_dict['kiwi'] = None

for i, text in tqdm(enumerate(ko_dict['word']), total=len(ko_dict)):
    ko_dict.at[i, 'kiwi'] = kiwi.tokenize(text)

100%|██████████| 1508680/1508680 [02:02<00:00, 12305.96it/s]


In [15]:
ko_dict.to_csv("ko_dict.csv", index=False)

In [24]:
# ko_dict = ko_dict[ko_dict['kiwi'].apply(lambda x: len(x) == 1)].reset_index(drop=True)
ko_dict['kiwi_pos' ] = ko_dict['kiwi'].apply(lambda x: x[0].tag)
# ko_dict[ko_dict['kiwi_pos'] == 'SL']
ko_dict

,word,freq,kiwi,kiwi_pos
0,내가,39801,"[(내가, NNG, 0, 4)]",NNG
1,그,35563,"[(그, NNG, 0, 2)]",NNG
2,난,35163,"[(난, NNG, 0, 3)]",NNG
3,내,33774,"[(내, NNG, 0, 2)]",NNG
4,안,32915,"[(안, NNG, 0, 3)]",NNG
...,...,...,...,...
1480627,흥창,0,"[(흥창, NNG, 0, 6)]",NNG
1480628,흥화공업,0,"[(흥화공업, NNG, 0, 11)]",NNG
1480629,희성건설,0,"[(희성건설, NNG, 0, 11)]",NNG
1480630,희성금속,0,"[(희성금속, NNG, 0, 11)]",NNG


In [ ]:
ko_dict_filtered = ko_dict[
    (ko_dict['pos_okt'].apply(len) == 1) & 
    (ko_dict['pos_okt'].apply(lambda x: x[0][1] == 'Noun')) &
    (ko_dict['word'].apply(lambda x: len(x) > 1)) &
    (ko_dict['freq'].apply(lambda x: x > 0))
]

ko_dict_filtered = (
    ko_dict_filtered.groupby('word', as_index=False)
    .agg({
        'freq': 'sum',  # Sum up the 'freq' column for duplicates
        **{col: 'first' for col in ko_dict_filtered.columns if col not in ['word', 'freq']}  # Keep the first occurrence for other columns
    })
)

In [ ]:
ko_dict_filtered = pd.read_csv("../data/korean_dict-hyunwoongko_korean-word-dict.csv")
ko_dict_filtered

In [ ]:
df = pd.read_csv("/home/hyujang/multilingual-inner-lexicon/data/r1_dataset_Babel-9B-Chat_Korean.csv")
df[df['label']=="realword"].sort_values(by='freq', ascending=False)

### German

In [ ]:
## German
# german_synsets = list(wordnet.all_synsets(lang="deu"))  # 'deu' is the ISO 639-3 code for German -> empty list

from xml.etree import ElementTree as ET
from lxml import etree
import re

def get_wordnet_lexicon_local(wnfile):
     loc_wn = open(wnfile,"r",encoding="utf-8")
     wntree = ET.parse(loc_wn)
     wnroot = wntree.getroot()
     lexicon = wnroot.find('Lexicon')
     return lexicon
 
def extract_all_nouns(wordnet_file):
    lexicon = get_wordnet_lexicon_local(wordnet_file)
    nouns = []
    for lexentry in lexicon.iter('LexicalEntry'):
        lemma = lexentry.find('Lemma')
        lemma_value = lemma.attrib['writtenForm']
        pos = lemma.attrib.get('partOfSpeech')
        if pos == 'n':
            nouns.append(lemma_value)
    return nouns
 
wordnet_file = '/work/hyujang/miniconda3/envs/thesis/lib/python3.12/site-packages/odenet/wordnet/deWordNet.xml'
nouns = extract_all_nouns(wordnet_file)
print(len(nouns))

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/olastor/german-word-frequencies/refs/heads/main/opensubtitles/opensubtitles_cistem_freq.csv"
df = pd.read_csv(url)
df.to_csv("opensubtitles_cistem_freq.csv", index=False)  # Save it locally if needed

## Wiki

In [ ]:
from kiwipiepy import Kiwi
# ko_noun_freq_df = pd.read_csv("../data/ko_wiki_noun_frequencies.csv")
# df = load_wikipedia_data(lang="ko")

kiwi = Kiwi()
# l = kiwi.tokenize(df['text'][0])
kiwi.tokenize("안녕하세요. 저는 한국어를 공부하고 있습니다.")

In [ ]:
import stanza
nlp = stanza.Pipeline(lang="de", processors="tokenize,pos,lemma")
text = "Angela Merkel besuchte das große Museum."
doc = nlp(text)
doc.sentences

In [ ]:
df_de = load_wikipedia_data("de")

In [ ]:
from datasets import load_dataset
import spacy
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import os
from nltk.stem import WordNetLemmatizer
import stanza

def load_wikipedia_data(lang, sample_size=20000):
    wiki = load_dataset("wikimedia/wikipedia", f"20231101.{lang}", split="train", columns=["text"])
    return wiki.shuffle(seed=2025).select(range(sample_size)).to_pandas()

def extract_nouns_with_frequency(text, lang, lemmatizer=None, nlp=None, kiwi=None):
    if lang == "en":
        """
        NN = Singular or mass noun
        NNS = Plural noun
        NNP = Singular proper noun (capitalized names)
        NNPS = Plural proper noun (capitalized plural names)
        """
        # doc = nlp(text)
        # nouns = [token.text for token in doc if token.pos_ == "NOUN"]
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        nouns = [lemmatizer.lemmatize(word.lower()) for word, tag in tagged if tag in ['NN', 'NNS']]
        nouns = [
            lemmatizer.lemmatize(word.lower()) 
            for word, tag in tagged 
            if tag in ['NN', 'NNS']
            # if tag in ['NN', 'NNS'] and not re.search(r'[\W\d]', word)
        ]
    elif lang == "de":
        doc = nlp(text)
        nouns = [
            word.lemma  # Use lemma instead of the original word
            for sentence in doc.sentences
            for word in sentence.words
            if word.upos == "NOUN"  # Filter only nouns
        ]
    elif lang == "ko":
        # nouns = okt.nouns(text)
        doc = kiwi.tokenize(text)
        nouns = [
            token.form 
            for token in doc 
            if token.tag == "NNG"
            # if token.tag == "NNG" and not re.search(r'[\W\d]', token.form)
        ]
    else:
        raise ValueError("Unsupported language")
    return Counter(nouns)

def process_wikipedia_nouns(lang):
    # global df
    
    output_file = f"../data/{lang}_wiki_noun_frequencies_lemmatized.csv"
    if os.path.exists(output_file):
        print(f"Output file '{output_file}' already exists. Skipping processing.")
        return  # Stop the function if the file exists

    print(f"Loading Wikipedia data for language: {lang}")
    # df = load_wikipedia_data(lang)
    df = df_de.iloc[:10000]
    
    if lang == "en":
        print("Extracting nouns using nltk for English...")
        lemmatizer = WordNetLemmatizer()
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, lemmatizer=lemmatizer))

    elif lang == "de":
        print(f"Extracting nouns using Stanza model for {lang}...")
        nlp = stanza.Pipeline(lang="de", processors="tokenize,pos,lemma", use_gpu=True)  # Initialize Stanza pipeline
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, nlp=nlp))

    elif lang == "ko":
        print(f"Extracting nouns using Okt tokenizer for {lang}...")
        # okt = Okt()
        kiwi = Kiwi()
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, kiwi=kiwi))
    
    print("Summing noun frequencies...")
    combined_noun_frequencies = sum((Counter(freq_dict) for freq_dict in tqdm(df["noun_frequencies"], desc="Summing")), Counter())
    noun_frequencies_df = pd.DataFrame.from_dict(combined_noun_frequencies, orient="index", columns=["frequency"])
    noun_frequencies_df.sort_values(by="frequency", ascending=False, inplace=True)
    noun_frequencies_df.reset_index(inplace=True)
    noun_frequencies_df.columns = ["word", "freq"]
    if lang == "en":
        noun_frequencies_df = noun_frequencies_df[~noun_frequencies_df['word'].str.contains(r'[^a-zA-Z]', na=False)]
        noun_frequencies_df.reset_index(inplace=True, drop=True)
        noun_frequencies_df.drop_duplicates(subset=['word'], inplace=True)
        noun_frequencies_df.dropna(inplace=True)
    if lang == "de":
        noun_frequencies_df = noun_frequencies_df[~noun_frequencies_df['word'].str.contains(r'[^a-zA-ZäöüÄÖÜß]', na=False)]
        noun_frequencies_df.reset_index(inplace=True, drop=True)
        noun_frequencies_df.drop_duplicates(subset=['word'], inplace=True)
        noun_frequencies_df.dropna(inplace=True)
    if lang == "ko":
        noun_frequencies_df = noun_frequencies_df[~noun_frequencies_df['word'].str.contains(r'[^\uac00-\ud7a3]', na=False)]
        noun_frequencies_df.reset_index(inplace=True, drop=True)
        noun_frequencies_df.drop_duplicates(subset=['word'], inplace=True)
        noun_frequencies_df.dropna(inplace=True)
    # output_file = f"../data/{lang}_wiki_noun_frequencies.csv"
    noun_frequencies_df.to_csv(output_file, index=False)
    print(f"Saved noun frequencies to {output_file}")


# process_wikipedia_nouns("en")
process_wikipedia_nouns("de")
# process_wikipedia_nouns("ko")

In [ ]:
from datasets import load_dataset
import spacy
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import os
from nltk.stem import WordNetLemmatizer
import stanza

def load_wikipedia_data(lang, sample_size=20000):
    wiki = load_dataset("wikimedia/wikipedia", f"20231101.{lang}", split="train", columns=["text"])
    return wiki.shuffle(seed=2025).select(range(sample_size)).to_pandas()

def extract_nouns_with_frequency(text, lang, lemmatizer=None, nlp=None, kiwi=None):
    if lang == "en":
        """
        NN = Singular or mass noun
        NNS = Plural noun
        NNP = Singular proper noun (capitalized names)
        NNPS = Plural proper noun (capitalized plural names)
        """
        # doc = nlp(text)
        # nouns = [token.text for token in doc if token.pos_ == "NOUN"]
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        nouns = [lemmatizer.lemmatize(word.lower()) for word, tag in tagged if tag in ['NN', 'NNS']]
        nouns = [
            lemmatizer.lemmatize(word.lower()) 
            for word, tag in tagged 
            if tag in ['NN', 'NNS']
            # if tag in ['NN', 'NNS'] and not re.search(r'[\W\d]', word)
        ]
    elif lang == "de":
        doc = nlp(text)
        nouns = [
            word.lemma  # Use lemma instead of the original word
            for sentence in doc.sentences
            for word in sentence.words
            if word.upos == "NOUN"  # Filter only nouns
        ]
    elif lang == "ko":
        # nouns = okt.nouns(text)
        doc = kiwi.tokenize(text)
        nouns = [
            token.form 
            for token in doc 
            if token.tag == "NNG"
            # if token.tag == "NNG" and not re.search(r'[\W\d]', token.form)
        ]
    else:
        raise ValueError("Unsupported language")
    return Counter(nouns)

def process_wikipedia_nouns(lang):
    # global df
    
    output_file = f"../data/{lang}_wiki_noun_frequencies_lemmatized.csv"
    if os.path.exists(output_file):
        print(f"Output file '{output_file}' already exists. Skipping processing.")
        return  # Stop the function if the file exists

    print(f"Loading Wikipedia data for language: {lang}")
    # df = load_wikipedia_data(lang)
    df = df_de.iloc[:10000]
    
    if lang == "en":
        print("Extracting nouns using nltk for English...")
        lemmatizer = WordNetLemmatizer()
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, lemmatizer=lemmatizer))

    elif lang == "de":
        print(f"Extracting nouns using Stanza model for {lang}...")
        nlp = stanza.Pipeline(lang="de", processors="tokenize,pos,lemma", use_gpu=True)  # Initialize Stanza pipeline
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, nlp=nlp))

    elif lang == "ko":
        print(f"Extracting nouns using Okt tokenizer for {lang}...")
        # okt = Okt()
        kiwi = Kiwi()
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, kiwi=kiwi))
    
    print("Summing noun frequencies...")
    combined_noun_frequencies = sum((Counter(freq_dict) for freq_dict in tqdm(df["noun_frequencies"], desc="Summing")), Counter())
    noun_frequencies_df = pd.DataFrame.from_dict(combined_noun_frequencies, orient="index", columns=["frequency"])
    noun_frequencies_df.sort_values(by="frequency", ascending=False, inplace=True)
    noun_frequencies_df.reset_index(inplace=True)
    noun_frequencies_df.columns = ["word", "freq"]
    if lang == "en":
        noun_frequencies_df = noun_frequencies_df[~noun_frequencies_df['word'].str.contains(r'[^a-zA-Z]', na=False)]
        noun_frequencies_df.reset_index(inplace=True, drop=True)
        noun_frequencies_df.drop_duplicates(subset=['word'], inplace=True)
        noun_frequencies_df.dropna(inplace=True)
    if lang == "de":
        noun_frequencies_df = noun_frequencies_df[~noun_frequencies_df['word'].str.contains(r'[^a-zA-ZäöüÄÖÜß]', na=False)]
        noun_frequencies_df.reset_index(inplace=True, drop=True)
        noun_frequencies_df.drop_duplicates(subset=['word'], inplace=True)
        noun_frequencies_df.dropna(inplace=True)
    if lang == "ko":
        noun_frequencies_df = noun_frequencies_df[~noun_frequencies_df['word'].str.contains(r'[^\uac00-\ud7a3]', na=False)]
        noun_frequencies_df.reset_index(inplace=True, drop=True)
        noun_frequencies_df.drop_duplicates(subset=['word'], inplace=True)
        noun_frequencies_df.dropna(inplace=True)
    # output_file = f"../data/{lang}_wiki_noun_frequencies.csv"
    noun_frequencies_df.to_csv(output_file, index=False)
    print(f"Saved noun frequencies to {output_file}")


# process_wikipedia_nouns("en")
process_wikipedia_nouns("de")
# process_wikipedia_nouns("ko")

In [ ]:
df_de.iloc[:10000]

In [ ]:
lang = "de"
noun_df = pd.read_csv(f"../data/{lang}_wiki_noun_frequencies_lemmatized.csv")
# noun_df = noun_df[~noun_df['word'].str.contains(r'[^a-zA-Z]', na=False)]
# noun_df.reset_index(inplace=True, drop=True)
# noun_df.to_csv(f"../data/{lang}_wiki_noun_frequencies_lemmatized.csv", index=False)
# noun_df[noun_df['word'].str.contains(r'[^a-zA-Z]', na=False)]
noun_df

In [ ]:
# noun_df.drop_duplicates(subset=['word'], inplace=True)
noun_df